In [41]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from datetime import datetime
import pickle

In [42]:
# Load the dataset from the /data folder
file_path = "../data/OnlineRetail.csv"  # Adjust path as needed
data = pd.read_csv(file_path, on_bad_lines='skip', encoding='ISO-8859-1')

In [43]:
# Display the first few rows to check
print("Dataset Loaded Successfully!")
display(data.head())


Dataset Loaded Successfully!


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


In [44]:
# Step 1: Remove rows with missing values in critical columns
# Removing rows where 'InvoiceDate' and 'CustomerID' have missing values
data = data.dropna(subset=['InvoiceDate', 'CustomerID'])

# Step 2: Label encode the 'CustomerID' and 'Country' columns
label_encoder = LabelEncoder()
data['CustomerID'] = label_encoder.fit_transform(data['CustomerID'].astype(str))
data['Country'] = label_encoder.fit_transform(data['Country'].astype(str))


In [45]:
# Step 3: Scale numerical features
numerical_cols = data.select_dtypes(include=['float64', 'int64']).columns
scaler = StandardScaler()
data[numerical_cols] = scaler.fit_transform(data[numerical_cols])


In [46]:
# Step 4: Convert 'InvoiceDate' to datetime format
data['InvoiceDate'] = pd.to_datetime(data['InvoiceDate'], errors='coerce')

In [10]:
# Step 5: Split the data into two transaction periods (6 months and next period)
tx_6m = data[(data['InvoiceDate'] < pd.Timestamp('2011-09-01')) & 
             (data['InvoiceDate'] >= pd.Timestamp('2011-03-01'))].reset_index(drop=True)

tx_next = data[(data['InvoiceDate'] >= pd.Timestamp('2011-09-01')) & 
               (data['InvoiceDate'] < pd.Timestamp('2011-12-01'))].reset_index(drop=True)


In [49]:
# Step 6: Create a user-level dataframe
tx_user = pd.DataFrame(tx_6m['CustomerID'].unique(), columns=['CustomerID'])

# Step 7: Get the last purchase for each customer in tx_6m
last_purchase = tx_6m.groupby('CustomerID')['InvoiceDate'].max().reset_index()
last_purchase.rename(columns={'InvoiceDate': 'LastPurchase'}, inplace=True)

# Step 8: Get the first purchase for each customer in tx_next
first_purchase = tx_next.groupby('CustomerID')['InvoiceDate'].min().reset_index()
first_purchase.rename(columns={'InvoiceDate': 'FirstPurchase'}, inplace=True)

# Step 9: Merge last and first purchases into tx_user
tx_user = tx_user.merge(last_purchase, on='CustomerID', how='left')
tx_user = tx_user.merge(first_purchase, on='CustomerID', how='left')

# Check for missing values after merging first and last purchase
print("Missing values in tx_user after merge:")
print(tx_user[['CustomerID', 'FirstPurchase', 'LastPurchase']].isnull().sum())

# Fill missing 'FirstPurchase' with a valid date (e.g., '1900-01-01')
tx_user['FirstPurchase'] = tx_user['FirstPurchase'].fillna(pd.Timestamp('1900-01-01'))  # Replace with a valid date
print(tx_user[['CustomerID', 'FirstPurchase', 'LastPurchase']].isnull().sum())


Missing values in tx_user after merge:
CustomerID          0
FirstPurchase    1113
LastPurchase        0
dtype: int64
CustomerID       0
FirstPurchase    0
LastPurchase     0
dtype: int64


In [50]:
# Step 10: Calculate the Target (difference in days between first and last purchase)
tx_user['Target'] = (tx_user['FirstPurchase'] - tx_user['LastPurchase']).dt.days

# Step 11: Fill missing 'Target' values with 999
tx_user['Target'] = tx_user['Target'].fillna(999).astype(int)


In [52]:
# Step 12: Feature Engineering: Aggregate features from tx_6m
tx_6m['TotalSpend'] = tx_6m['Quantity'] * tx_6m['UnitPrice']

user_features = tx_6m.groupby('CustomerID').agg({
    'Quantity': ['sum', 'mean'],
    'TotalSpend': ['sum', 'mean'],
    'InvoiceDate': ['count']
}).reset_index()

# Rename columns for user features
user_features.columns = ['CustomerID', 'QuantitySum', 'QuantityMean', 'TotalSpendSum', 'TotalSpendMean', 'PurchaseCount']

# Rename the columns for better readability
user_features.columns = ['CustomerID', 'QuantitySum', 'QuantityMean', 'TotalSpendSum', 'TotalSpendMean', 'PurchaseCount']

# Step 2: Fill missing aggregated values with zero or NaN
# If 'QuantitySum' has missing values, fill with 0 (or NaN, depending on what you want to do)
user_features['QuantitySum'] = user_features['QuantitySum'].fillna(0)

# Step 3: Merge user features with tx_user
tx_user = tx_user.merge(user_features, on='CustomerID', how='left')

# Step 4: Verify after merge to check for any remaining missing values
print(tx_user.isnull().sum())

# Step 13: Calculate Recency (days since last purchase)
tx_user['Recency'] = (pd.Timestamp('2011-09-01') - tx_user['LastPurchase']).dt.days

# Step 14: Create a new DataFrame for feature encoding (StockCode, etc.)
tx_features = tx_6m[['CustomerID', 'InvoiceDate', 'Quantity', 'UnitPrice', 'StockCode']].copy()

# Step 15: One-hot encode the 'StockCode' column
tx_features = pd.get_dummies(tx_features, columns=['StockCode'], drop_first=True)

# Step 16: Label encode 'CustomerID' in tx_features DataFrame
tx_features['CustomerID'] = label_encoder.fit_transform(tx_features['CustomerID'].astype(str))


CustomerID          0
LastPurchase        0
FirstPurchase       0
Target              0
QuantitySum_x       0
QuantityMean_x      0
TotalSpendSum_x     0
TotalSpendMean_x    0
PurchaseCount_x     0
Recency             0
QuantitySum_y       0
QuantityMean_y      0
TotalSpendSum_y     0
TotalSpendMean_y    0
PurchaseCount_y     0
dtype: int64


In [53]:
# Step 17: Check data types, missing values, and duplicates in key columns
print(tx_user.dtypes)
print(user_features.dtypes)
print(tx_features.dtypes)

# Check for missing values
print(tx_user['CustomerID'].isnull().sum())
print(user_features['CustomerID'].isnull().sum())
print(tx_features['CustomerID'].isnull().sum())

# Check for duplicates in 'CustomerID'
print(tx_user['CustomerID'].duplicated().sum())
print(user_features['CustomerID'].duplicated().sum())
print(tx_features['CustomerID'].duplicated().sum())


CustomerID                 float64
LastPurchase        datetime64[ns]
FirstPurchase       datetime64[ns]
Target                       int64
QuantitySum_x              float64
QuantityMean_x             float64
TotalSpendSum_x            float64
TotalSpendMean_x           float64
PurchaseCount_x              int64
Recency                      int64
QuantitySum_y              float64
QuantityMean_y             float64
TotalSpendSum_y            float64
TotalSpendMean_y           float64
PurchaseCount_y              int64
dtype: object
CustomerID        float64
QuantitySum       float64
QuantityMean      float64
TotalSpendSum     float64
TotalSpendMean    float64
PurchaseCount       int64
dtype: object
CustomerID                  int64
InvoiceDate        datetime64[ns]
Quantity                  float64
UnitPrice                 float64
StockCode_10080              bool
                        ...      
StockCode_D                  bool
StockCode_DOT                bool
StockCode_M        

In [54]:
# Step 18: Merge user features and transaction features

# Convert CustomerID to int64 again to ensure there are no implicit floats
tx_user['CustomerID'] = tx_user['CustomerID'].astype(int)
tx_features['CustomerID'] = tx_features['CustomerID'].astype(int)

# Drop rows with NaN in CustomerID if necessary
tx_user = tx_user.dropna(subset=['CustomerID'])
tx_features = tx_features.dropna(subset=['CustomerID'])

# Step 1: Simplified merge for user features
user_features = user_features[['CustomerID', 'QuantitySum', 'QuantityMean', 'TotalSpendSum', 'TotalSpendMean', 'PurchaseCount']]
final_features = tx_user.merge(user_features, on='CustomerID', how='left')

# Step 2: Simplified merge for transaction features (tx_features)
final_features = final_features.merge(tx_features[['CustomerID'] + [col for col in tx_features.columns if col != 'CustomerID']], on='CustomerID', how='inner')

# Step 3: Check the result of the merge
print(final_features.head())
print(final_features.shape) 

/tmp/ipykernel_109744/311999628.py:13: UserWarning: You are merging on int and float columns where the float values are not equal to their int representation.
  final_features = tx_user.merge(user_features, on='CustomerID', how='left')


   CustomerID        LastPurchase FirstPurchase  Target  QuantitySum_x  \
0           0 2011-05-19 14:14:00    1900-01-01  -40681      -0.699011   
1           0 2011-05-19 14:14:00    1900-01-01  -40681      -0.699011   
2           0 2011-05-19 14:14:00    1900-01-01  -40681      -0.699011   
3           0 2011-05-19 14:14:00    1900-01-01  -40681      -0.699011   
4           0 2011-05-19 14:14:00    1900-01-01  -40681      -0.699011   

   QuantityMean_x  TotalSpendSum_x  TotalSpendMean_x  PurchaseCount_x  \
0         -0.0233         -0.00682         -0.000227               30   
1         -0.0233         -0.00682         -0.000227               30   
2         -0.0233         -0.00682         -0.000227               30   
3         -0.0233         -0.00682         -0.000227               30   
4         -0.0233         -0.00682         -0.000227               30   

   Recency  ...  StockCode_90214V  StockCode_90214Y  StockCode_BANK CHARGES  \
0      104  ...             False    

In [36]:
# Step 19: Split data into features (X) and target (y)
if 'Target' not in final_features.columns:
    print("Target column is missing. Recheck the target calculation.")
else:
    X = final_features.drop(columns=['Target'])  # Features
    y = final_features['Target']  # Target (days until next purchase)
    print(f"Feature columns: {X.columns}")


Feature columns: Index(['CustomerID', 'LastPurchase', 'FirstPurchase', 'Recency', 'QuantitySum',
       'QuantityMean', 'TotalSpendSum', 'TotalSpendMean', 'PurchaseCount',
       'InvoiceDate',
       ...
       'StockCode_90214V', 'StockCode_90214Y', 'StockCode_BANK CHARGES',
       'StockCode_C2', 'StockCode_CRUK', 'StockCode_D', 'StockCode_DOT',
       'StockCode_M', 'StockCode_PADS', 'StockCode_POST'],
      dtype='object', length=3211)


In [37]:
# Step 20: Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

In [38]:
# Print shapes of the resulting datasets
print(f"Training set size: {X_train.shape[0]} samples")
print(f"Testing set size: {X_test.shape[0]} samples")


Training set size: 105166 samples
Testing set size: 26292 samples


In [39]:
# Check for missing values in the features after the split
print(X_train.isnull().sum())  # Check missing values in training data
print(X_test.isnull().sum())   # Check missing values in testing data


CustomerID             0
LastPurchase           0
FirstPurchase          0
Recency                0
QuantitySum       105166
                   ...  
StockCode_D            0
StockCode_DOT          0
StockCode_M            0
StockCode_PADS         0
StockCode_POST         0
Length: 3211, dtype: int64
CustomerID            0
LastPurchase          0
FirstPurchase         0
Recency               0
QuantitySum       26292
                  ...  
StockCode_D           0
StockCode_DOT         0
StockCode_M           0
StockCode_PADS        0
StockCode_POST        0
Length: 3211, dtype: int64


In [149]:
import pickle

# Save the datasets to a file
with open('train_test_data.pkl', 'wb') as f:
    pickle.dump((X_train, y_train, X_test, y_test), f)

print("Data saved successfully!")


Data saved successfully!
